In [106]:
import pandas as pd

import numpy as np

from nltk.corpus import stopwords

from pymorphy3 import MorphAnalyzer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV

In [107]:
df_train=pd.read_csv("clean_train.csv")
df_test=pd.read_csv("clean_test.csv")

In [108]:
df_train.dtypes

id                                 int64
comment                           object
rating                             int64
Вопрос не решен                    int64
Вопрос решен                       int64
Не понравилось качество услуги     int64
Не понравился результат услуги     int64
Понравилась работа сотрудников     int64
Понравилась скорость работы        int64
Понравилось качество услуги        int64
Понравился результат услуги        int64
Претензии и предложения            int64
dtype: object

In [131]:
tag_columns=["Вопрос не решен",
             "Вопрос решен",
             "Не понравилось качество услуги",
             "Не понравился результат услуги",
             "Понравилась работа сотрудников",
             "Понравилась скорость работы",
             "Понравилось качество услуги",
             "Понравился результат услуги",
             "Претензии и предложения"]

## препроцессинг

### лемур хз не читал

In [132]:
morph=MorphAnalyzer()
def lemmatize_text(text):
    words=text.split()
    return ' '.join([morph.parse(word)[0].normal_form for word in words])

df_train['comment']=df_train['comment'].apply(lemmatize_text)
df_test['comment']=df_test['comment'].apply(lemmatize_text)

### веса для тэгов

In [133]:
def calculate_class_weights(y):
    class_weights=[]
    for tag in y.columns:
        pos=np.sum(y[tag])
        neg=len(y) - pos
        class_weights.append({0:1,1:neg/pos if pos>0 else 1})
    return class_weights

y=df_train[tag_columns]
class_weights=calculate_class_weights(y)

### слова типа ну э ам ну как там ну типа это ну кароче ну вы поняли

In [134]:
russian_stop_words=stopwords.words('russian')
preprocessor=ColumnTransformer(
    transformers=[
        ('text',TfidfVectorizer(
            stop_words=russian_stop_words,
            max_features=5000,
            ngram_range=(1,2)),
        'comment'),
        ('num',StandardScaler(),['rating'])],
    remainder='drop')

X_train=df_train[['comment','rating']]
y_train=df_train[tag_columns]

X_test=df_test[['comment','rating']]
y_test=df_test[tag_columns]

## Модели

### логхистичшеска рэгхрэсся

In [ ]:
model=Pipeline([
    ('preprocessor',preprocessor),
    ('clf',MultiOutputClassifier(
        LogisticRegression(
            max_iter=1000,
            class_weight='balanced')))])

### случайный лес

In [135]:
model=Pipeline([
    ('preprocessor',preprocessor),
    ('clf',MultiOutputClassifier(
        RandomForestClassifier(
            class_weight='balanced_subsample',
            n_estimators=100)))])

## тренировка и оценочка

### тренировка

In [150]:
model.fit(X_train, y_train)
y_proba_train=model.predict_proba(X_train)
y_pred_train=model.predict(X_train)
y_proba_test=model.predict_proba(X_test)
y_pred_test=model.predict(X_test)

### поиск клеточек

In [151]:
param_grid = {
    'preprocessor__text__max_features':[3000,5000,7000],
    'preprocessor__text__ngram_range':[(1,1),(1,2)],
    'clf__estimator__C':[0.1,1,10],}
model = Pipeline([
    ('preprocessor',preprocessor),
    ('clf',OneVsRestClassifier(LogisticRegression(max_iter=1000,class_weight='balanced')))])
grid_search = GridSearchCV(
    model,
    param_grid,
    scoring='roc_auc_ovr',
    cv=3,
    n_jobs=-1,
    verbose=1)
grid_search.fit(X_train,y_train)
best_model = grid_search.best_estimator_
print("Best parameters:",grid_search.best_params_)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
Best parameters: {'clf__estimator__C': 1, 'preprocessor__text__max_features': 3000, 'preprocessor__text__ngram_range': (1, 1)}


### оценочка

In [154]:
roc_auc_scores={}
f1_scores={}
for i,tag in enumerate(tag_columns):
    roc_auc_train=round(roc_auc_score(y_train[tag],y_proba_train[i][:,1]),2)
    roc_auc_test=round(roc_auc_score(y_test[tag],y_proba_test[i][:,1]),2)

    f1_train_macro=round(f1_score(y_train[tag],y_pred_train[:,i],average='macro'),3)
    f1_test_macro=round(f1_score(y_test[tag],y_pred_test[:,i],average='macro'),3)
    f1_train_weighted=round(f1_score(y_train[tag],y_pred_train[:,i],average='weighted'),3)
    f1_test_weighted=round(f1_score(y_test[tag],y_pred_test[:,i],average='weighted'),3)

    print(f"tag: {tag:<40} roc-auc train: {roc_auc_train}roc-auc test: {roc_auc_test}")

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

## жестокий дебагинг

In [145]:
def predict_tags_with_proba(text,rating,threshold=0.5):
    new_data=pd.DataFrame({'comment':[text],'rating':[rating]})
    probabilities=best_model.predict_proba(new_data)
    results=[]
    for i,tag in enumerate(tag_columns):
        proba=probabilities[0][i]
        results.append({
            'tag':tag,
            'probability':proba,
            'prediction':proba>=threshold})
    results.sort(key=lambda x:x['probability'],reverse=True)
    return results

In [147]:
new_comment="сотрудник быстро решил мой вопрос, качество услуги на высоте!"
new_rating=5
predictions=predict_tags_with_proba(new_comment,new_rating)
print("Predicted Tags:")
for pred in predictions:
    print(f"{pred['tag']:<40} {pred['probability']:.3f} ({'✓' if pred['prediction'] else '✗'})")

Predicted Tags:
Понравилась работа сотрудников           0.948 (✓)
Понравилось качество услуги              0.791 (✓)
Понравился результат услуги              0.727 (✓)
Понравилась скорость работы              0.694 (✓)
Вопрос решен                             0.559 (✓)
Вопрос не решен                          0.364 (✗)
Претензии и предложения                  0.278 (✗)
Не понравился результат услуги           0.103 (✗)
Не понравилось качество услуги           0.100 (✗)
